In [1]:
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import h5py
import napari
import tifffile as tiff
import seaborn as sns
import pickle

import networkx as nx
from sklearn.model_selection import train_test_split

In [2]:
import torch_geometric.utils
import torch

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()

In [6]:
import graph
import torch
import torch_geometric.utils
import networkx as nx
import lightning.pytorch as pl
import torch.utils.data as data

spatial_omics_folder = (Path().cwd().parents[0]).absolute() / 'data' / 'spatial_omics_graph'
process_path = (Path().cwd().parents[0]).absolute() / 'data' / 'torch_graph_data'

# Create data loader

In [7]:
from torch_geometric.loader import DataLoader
seed = torch.Generator().manual_seed(42)

name = 'NIH_pos'

# Crate dataset
dataset = graph.GraphDatasetPos(process_path / name, process_path / name / 'info.csv', 2, y_name='covid')

train_set, val_set, test_set = graph.train_test_val_split(dataset)

# Create Dataloader
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)


In [8]:
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.nc}')

Dataset: GraphDatasetPos(442):
Number of graphs: 442
Number of features: 6
Number of classes: 2


In [9]:
print(f'Train set: {len(train_set)}, val set: {len(test_set)}, test set: {len(val_set)}')

Train set: 213, val set: 176, test set: 53


In [10]:
for step, data in enumerate(test_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()
    data.label
    break

Step 1:
Number of graphs in the current batch: 32
DataBatch(edge_index=[2, 653260], num_nodes=93748, x=[93748, 6], pos=[93748, 2], node_types=[93748], label=[32], covid=[32], train_mask=[93748], test_mask=[93748], edge_attr=[653260, 2], y=[32], name=[32], batch=[93748], ptr=[33])



# Train network K Fold

In [11]:
from lightning.pytorch.accelerators import find_usable_cuda_devices
import wandb

In [12]:
from sklearn.model_selection import KFold, StratifiedKFold, ShuffleSplit, StratifiedShuffleSplit
from torch.utils.data import SubsetRandomSampler

In [13]:
# # Print K fold model number of samples and number of positive cases

# k_folds = 5
# kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# for fold, (train_ids, valid_ids) in enumerate(kfold.split(dataset)):
#     train_subset = dataset.index_select(train_ids.tolist())
#     val_subset = dataset.index_select(valid_ids.tolist())
    
#     train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
#     val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)
    
#     train_positives = 0
#     test_positives = 0
#     for step, data in enumerate(train_loader):
#         train_positives += np.sum(data.y.numpy())
#     for step, data in enumerate(val_loader):
#         test_positives += np.sum(data.y.numpy())
        
#     print(len(train_subset), train_positives, len(val_subset), test_positives)

In [14]:
condition = '082923_Covid_pos'
checkpoint_folder = (Path().cwd().parents[0]).absolute() / 'data' / "saved_models" / f"Graph_GNNs_{condition}" 
project_name = f'SF_{condition}'

In [15]:
AVAIL_GPUS = [0]

# Setting the seed
pl.seed_everything(42)

NUM_LAYERS = 2
HIDDEN_CHANNELS = 16
pools = ['mean', 'max', 'attention', 'attention2']
# pools = ['attention2']
models = ['GAT', 'GINConv']
# models = ['GINConv']
epochs =100

Global seed set to 42


In [16]:
ys = [data.y.item() for data in dataset]

In [17]:
# Print K fold model number of samples and number of positive cases

k_folds = 5
# kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)
kfold = StratifiedShuffleSplit(n_splits=k_folds, test_size=0.3, random_state=0)

for fold, (train_ids, valid_ids) in enumerate(kfold.split(dataset, ys)):
    train_subset = dataset.index_select(train_ids.tolist())
    val_subset = dataset.index_select(valid_ids.tolist())
    
    for pool in pools:
        for model in models:
            if model == 'GAT':
                batch_size=64
            else:
                batch_size=128
            # Path to the folder where the pretrained models are saved
            CHECKPOINT_PATH = checkpoint_folder / f'{model}_{NUM_LAYERS}_{HIDDEN_CHANNELS}_{fold}' / pool
            CHECKPOINT_PATH.mkdir(parents=True, exist_ok=True)

            # Skip already trained kfold and pool
            checkpoint = CHECKPOINT_PATH / f"GraphLevel{model}" / f"GraphLevel{model}.ckpt" 
            if checkpoint.exists():
                print(checkpoint)
                continue

            # Run training
            run = wandb.init(project=project_name, name=f'{model}_{NUM_LAYERS}_{HIDDEN_CHANNELS}_{fold}', 
                            group=f'{model}_{pool}')
            graph.train_graph_classifier_kfold(model, 
                                                 train_subset, 
                                                 val_subset, 
                                                 dataset, 
                                                 CHECKPOINT_PATH, 
                                                 AVAIL_GPUS, 
                                                 hidden_channels=HIDDEN_CHANNELS, 
                                                 num_layers=NUM_LAYERS, 
                                                 epochs=epochs,
                                                 batch_size=batch_size,
                                                 graph_pooling=pool,
                                                pos=True)
            run.finish()


Y:\coskun-lab\Thomas\11_snowflakes\data\saved_models\Graph_GNNs_082923_Covid_pos\GAT_2_16_0\mean\GraphLevelGAT\GraphLevelGAT.ckpt


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: thoomas. Use `wandb login --relogin` to force relogin


Global seed set to 42
C:\Users\thu71\Anaconda3\envs\snowflake\lib\site-packages\pytorch_lightning\loggers\wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

   | Name        | Type             | Params
--------------------------------------------------
0  | model       | GIN              | 1.0 K 
1  | fnn_layer   | Linear           | 272   
2  | selu        | SELU             | 0     
3  | head        | Linear           | 34    
4  | loss_module | CrossEntropyLoss | 0     
5  | train_acc   | BinaryAccuracy   | 0     
6  | train_auroc | BinaryAUROC      | 0     
7  | train_f1    | BinaryF1Sco

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▄▅▅▆▆▅▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇█▇▇▇███▇▇▇▇▇████▇█
train_auc,▁▄▆▆▇▇▆▇▇▇▇▇▇▇▇█▇▇▇▇█▇██████████████████
train_f1,▁▃▄▅▅▅▅▆▆▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇███▇▇▇▇▇███▇▇▇
train_loss_epoch,█▆▄▄▃▃▄▃▃▃▂▂▂▃▃▂▂▃▃▂▂▂▁▁▂▁▁▁▁▂▂▂▁▂▁▁▁▂▂▁
train_loss_step,█▅▅▅▃▄▃▃▂▃▄▃▃▃▁▂▂▂▂▁▃▂▁▂▂▂▄▁▂▂
trainer/global_step,▁▁▁▂▁▂▂▂▃▃▃▃▃▃▄▄▄▄▅▃▄▄▅▅▆▄▄▄▆▇▇▅▅▅███▆▆▆
val_acc,▁▂▂▄▆▆▆▆▅▆▇▆▆▇▆▆▆▅▇▇▆▆▆▆▇▇▇▆▆█▆▇██▆▆▇▇▆▇
val_auc,▁▃▇▇▇▇▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██▇█▇▇▇████████
val_f1,▁▂▃▄▆▆▆▆▄▅▇▆▆▆▆▆▆▅▆▆▄▆▆▄▆▆▇▅▅█▅▆██▄▅▇▇▅▆


Y:\coskun-lab\Thomas\11_snowflakes\data\saved_models\Graph_GNNs_082923_Covid_pos\GAT_2_16_0\max\GraphLevelGAT\GraphLevelGAT.ckpt
Y:\coskun-lab\Thomas\11_snowflakes\data\saved_models\Graph_GNNs_082923_Covid_pos\GINConv_2_16_0\max\GraphLevelGINConv\GraphLevelGINConv.ckpt
Y:\coskun-lab\Thomas\11_snowflakes\data\saved_models\Graph_GNNs_082923_Covid_pos\GAT_2_16_0\attention\GraphLevelGAT\GraphLevelGAT.ckpt
Y:\coskun-lab\Thomas\11_snowflakes\data\saved_models\Graph_GNNs_082923_Covid_pos\GINConv_2_16_0\attention\GraphLevelGINConv\GraphLevelGINConv.ckpt
Y:\coskun-lab\Thomas\11_snowflakes\data\saved_models\Graph_GNNs_082923_Covid_pos\GAT_2_16_0\attention2\GraphLevelGAT\GraphLevelGAT.ckpt
Y:\coskun-lab\Thomas\11_snowflakes\data\saved_models\Graph_GNNs_082923_Covid_pos\GINConv_2_16_0\attention2\GraphLevelGINConv\GraphLevelGINConv.ckpt
Y:\coskun-lab\Thomas\11_snowflakes\data\saved_models\Graph_GNNs_082923_Covid_pos\GAT_2_16_1\mean\GraphLevelGAT\GraphLevelGAT.ckpt


...

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▃▅▅▅▆▆▆▆▆▇▇▇▇▆▇▇▇▇▇▇██▇▇▇▇▇▇▇█▇████████
train_auc,▁▄▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇███████████████████████
train_f1,▁▂▄▄▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇█▇▇▇▇▇▇▇▇▇▇█▇███▇▇█
train_loss_epoch,█▆▄▄▄▄▃▃▃▃▂▂▃▃▃▃▂▂▂▂▂▂▂▂▁▂▂▂▁▂▂▁▁▁▁▁▁▁▂▁
train_loss_step,█▇▄▅▆▄▄▅▅▄▅▄▃▂▂▂▃▂▂▂▃▃▃▃▂▂▃▁▂▁
trainer/global_step,▁▁▁▂▁▂▂▂▃▃▃▃▃▃▄▄▄▄▅▃▄▄▅▅▆▄▄▄▆▇▇▅▅▅███▆▆▆
val_acc,▁▂▂▃▄▅▅▅▆▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇█▇▇▆███▇▇██▇▇▇█▇
val_auc,▁▂▄▅▆▆▆▆▇▇▇▇▇▇▇▇▇█▇█▇█▇▇█▇█▇██▇█████████
val_f1,▂▁▃▃▄▄▄▃▆▆▆▅▅▇▆▇▆▇▇▇▇▇▇▇█▇▆▅█▇█▇▇██▇▇▇█▆


Y:\coskun-lab\Thomas\11_snowflakes\data\saved_models\Graph_GNNs_082923_Covid_pos\GAT_2_16_3\max\GraphLevelGAT\GraphLevelGAT.ckpt
Y:\coskun-lab\Thomas\11_snowflakes\data\saved_models\Graph_GNNs_082923_Covid_pos\GINConv_2_16_3\max\GraphLevelGINConv\GraphLevelGINConv.ckpt
Y:\coskun-lab\Thomas\11_snowflakes\data\saved_models\Graph_GNNs_082923_Covid_pos\GAT_2_16_3\attention\GraphLevelGAT\GraphLevelGAT.ckpt
Y:\coskun-lab\Thomas\11_snowflakes\data\saved_models\Graph_GNNs_082923_Covid_pos\GINConv_2_16_3\attention\GraphLevelGINConv\GraphLevelGINConv.ckpt
Y:\coskun-lab\Thomas\11_snowflakes\data\saved_models\Graph_GNNs_082923_Covid_pos\GAT_2_16_3\attention2\GraphLevelGAT\GraphLevelGAT.ckpt
Y:\coskun-lab\Thomas\11_snowflakes\data\saved_models\Graph_GNNs_082923_Covid_pos\GINConv_2_16_3\attention2\GraphLevelGINConv\GraphLevelGINConv.ckpt
Y:\coskun-lab\Thomas\11_snowflakes\data\saved_models\Graph_GNNs_082923_Covid_pos\GAT_2_16_4\mean\GraphLevelGAT\GraphLevelGAT.ckpt


Global seed set to 42
C:\Users\thu71\Anaconda3\envs\snowflake\lib\site-packages\pytorch_lightning\loggers\wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

   | Name        | Type             | Params
--------------------------------------------------
0  | model       | GIN              | 1.0 K 
1  | fnn_layer   | Linear           | 272   
2  | selu        | SELU             | 0     
3  | head        | Linear           | 34    
4  | loss_module | CrossEntropyLoss | 0     
5  | train_acc   | BinaryAccuracy   | 0     
6  | train_auroc | BinaryAUROC      | 0     
7  | train_f1    | BinaryF1Sco

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▃▅▅▆▆▆▆▇▇▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇██▇██▇▇█▇█▇███▇
train_auc,▁▅▆▇▇▇▇▇▇▇▇▇▇▇▇█▇█▇█████████████████████
train_f1,▁▂▄▄▅▅▅▅▆▆▅▆▆▆▆▇▆▇▆▆▇▇▇▆▆▇█▇█▇▇▇▇▇▇▇█▇▇▇
train_loss_epoch,█▇▅▄▄▄▃▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂▁▁▁▂▁▁▁▂▁▂
train_loss_step,█▆▄▄▅▅▃▄▅▄▃▃▃▂▁▂▁▁▁▂▃▂▂▂▃▂▁▂▂▂
trainer/global_step,▁▁▁▂▁▂▂▂▃▃▃▃▃▃▄▄▄▄▅▃▄▄▅▅▆▄▄▄▆▇▇▅▅▅███▆▆▆
val_acc,▁▄▃▄▅▆▆▅▅▆▆▆▆▆▅▆▆▅▇▆▅▇▇▇▇▇▇▇██▇▆▇██▇█▇▇▇
val_auc,▁▄▆▆▅▅▅▄▅▆▅▆▆▇▆▇▇▇▇▇▇█▇▇▇▇▇▇██▇▆▇▇█▇▇█▇█
val_f1,▁▄▃▄▅▆▅▄▃▄▅▅▅▆▄▅▆▅▆▆▅▇▇▆▇▇▇▆██▇▅▇██▆▇▇▇▇


Y:\coskun-lab\Thomas\11_snowflakes\data\saved_models\Graph_GNNs_082923_Covid_pos\GAT_2_16_4\max\GraphLevelGAT\GraphLevelGAT.ckpt
Y:\coskun-lab\Thomas\11_snowflakes\data\saved_models\Graph_GNNs_082923_Covid_pos\GINConv_2_16_4\max\GraphLevelGINConv\GraphLevelGINConv.ckpt
Y:\coskun-lab\Thomas\11_snowflakes\data\saved_models\Graph_GNNs_082923_Covid_pos\GAT_2_16_4\attention\GraphLevelGAT\GraphLevelGAT.ckpt
Y:\coskun-lab\Thomas\11_snowflakes\data\saved_models\Graph_GNNs_082923_Covid_pos\GINConv_2_16_4\attention\GraphLevelGINConv\GraphLevelGINConv.ckpt
Y:\coskun-lab\Thomas\11_snowflakes\data\saved_models\Graph_GNNs_082923_Covid_pos\GAT_2_16_4\attention2\GraphLevelGAT\GraphLevelGAT.ckpt
Y:\coskun-lab\Thomas\11_snowflakes\data\saved_models\Graph_GNNs_082923_Covid_pos\GINConv_2_16_4\attention2\GraphLevelGINConv\GraphLevelGINConv.ckpt
